In [1]:
import torch
from transformers import CLIPProcessor, CLIPModel
from torch.utils.data import Dataset, DataLoader
from PIL import Image

/mnt/d/gery/clip/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
dataset = [
    ("data/app-launch.png", "Illustrasi popsy"),
    ("data/shaking-hands.png", "Ilustrasi popsy"),
]

In [34]:
from torch.utils.data import Dataset
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset 
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image_path, text = self.dataset[idx]  # Ambil path gambar dan teks
        image = Image.open(image_path).convert("RGB")  # Pastikan gambar dalam format RGB
        inputs = self.processor(text=text, images=image, return_tensors="pt", padding=True)
        return inputs


In [35]:
# SETUP TRAINING
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# load data
train_dataset = CustomDataset(dataset, processor)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [36]:
%ls data

app-launch.png*  shaking-hands.png*  work-party.png*


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [37]:
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def process_image(image_path):
    image = Image.open(image_path)
    inputs = processor(images=image, return_tensors="pt", padding=True)
    return inputs

image = "data/app-launch.png"
inputs = process_image(image)
print(inputs['pixel_values'].shape)


torch.Size([1, 3, 224, 224])


In [43]:
for epoch in range(3):
    for batch in train_dataloader:
        pixel_values = batch['pixel_values'].squeeze(1)  # Mengambil nilai pixel dari gambar
        input_ids = batch['input_ids'].squeeze(1)  # Mengambil input_ids dari teks

        pixel_values = pixel_values.to(model.device)
        input_ids = input_ids.to(model.device)

        # Memastikan return_loss=True
        outputs = model(input_ids=input_ids, pixel_values=pixel_values, return_loss=True)

        # Periksa apakah 'loss' ada dalam output
        if outputs.loss is not None:
            print(outputs.loss)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            print(f"Epoch {epoch + 1}, Loss: {loss.item()}")
        else:
            print(f"No loss returned for this batch in Epoch {epoch + 1}")


tensor(0.9376, grad_fn=<DivBackward0>)
Epoch 1, Loss: 0.9376348853111267
tensor(1.2747, grad_fn=<DivBackward0>)
Epoch 2, Loss: 1.2746667861938477
tensor(0.1322, grad_fn=<DivBackward0>)
Epoch 3, Loss: 0.13217653334140778


In [45]:
model.save_pretrained("saved_model")

In [47]:
processor.save_pretrained("saved_model")

[]

In [48]:
model = CLIPModel.from_pretrained("saved_model")
processor = CLIPProcessor.from_pretrained("saved_model")

model.eval()

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

In [51]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

model = CLIPModel.from_pretrained("saved_model")

processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

model.eval()

image_path = "data/engineer.png"
text = ["Illustration popsy", "person"]

image = Image.open(image_path)

inputs = processor(text=text, images=image, return_tensors="pt", padding=True)

with torch.no_grad(): 
    outputs = model(**inputs)

logits_per_image = outputs.logits_per_image 
logits_per_text = outputs.logits_per_text 

probs = logits_per_image.softmax(dim=1)

print(f"Prediksi probabilitas untuk gambar terhadap teks: {probs}")


Prediksi probabilitas untuk gambar terhadap teks: tensor([[0.8071, 0.1929]])
